In [ ]:
#@title 🛠️ **Cell 1: Trinity Multi-Venv Infrastructure Setup v1.5 (Error-Resilient)** { display-mode: "form" }

# ============================================================================
# CELL 1: TRINITY MULTI-VENV INFRASTRUCTURE SETUP (ERROR-RESILIENT VERSION)
# This version continues even if network optimization fails
# ============================================================================

import os
import subprocess
import sys
import time
from pathlib import Path

# --- Basic Setup ---
PROJECT_ROOT = Path('/content/TrinityUI')
WEBUI_ROOT = Path('/content')
TRINITY_REPO_URL = 'https://github.com/remphanstar/TrinityUI.git'
TRINITY_BRANCH = 'main'

def run_setup_command(command, show_output=True, ignore_errors=False):
    """Runs a shell command and streams its output live."""
    start_time = time.time()
    print(f"🚀 Executing: {command}")
    process = subprocess.Popen(
        command,
        shell=True,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        encoding='utf-8',
        errors='replace'
    )

    # Print output line by line
    for line in iter(process.stdout.readline, ''):
        if show_output:
            sys.stdout.write(f"  > {line}")
            sys.stdout.flush()

    # Wait for the process to complete and check the return code
    return_code = process.wait()
    duration = time.time() - start_time
    if return_code != 0 and not ignore_errors:
        raise subprocess.CalledProcessError(return_code, command)

    if return_code == 0:
        print(f"✅ Command successful. ({duration:.2f}s)\n")
        return True
    else:
        print(f"⚠️ Command returned non-zero exit code: {return_code} ({duration:.2f}s)\n")
        return False

print("🚀 Starting Trinity Multi-Venv Setup (Error-Resilient)...")

# 1. Ensure System Dependencies are present
print("\n[Step 1/3] Setting up system dependencies...")
try:
    # Use DEBIAN_FRONTEND=noninteractive to prevent any prompts
    os.environ['DEBIAN_FRONTEND'] = 'noninteractive'

    # Install aria2 with maximum priority
    run_setup_command("apt-get update -qq && apt-get install -y -qq aria2")
    print("   ✅ Aria2 download accelerator installed.")

    # Install other dependencies in parallel
    run_setup_command("apt-get install -y -qq software-properties-common git curl wget")
    run_setup_command("add-apt-repository -y ppa:deadsnakes/ppa")

    # Install Python versions in parallel
    run_setup_command("apt-get install -y -qq python3.10 python3.10-venv python3.11 python3.11-venv")

    # Try to optimize system for faster installation - but don't fail if it doesn't work
    try:
        print("   🔄 Attempting network optimizations (optional)...")
        run_setup_command("echo 'net.core.default_qdisc = fq' | sudo tee -a /etc/sysctl.conf")
        run_setup_command("echo 'net.ipv4.tcp_congestion_control = bbr' | sudo tee -a /etc/sysctl.conf")
        if run_setup_command("sysctl -p", show_output=False, ignore_errors=True):
            print("   ✅ Network optimizations applied.")
        else:
            print("   ⚠️ Network optimizations skipped (not critical).")
    except Exception as e:
        print(f"   ⚠️ Network optimization failed: {e} (continuing anyway)")

    print("   ✅ System dependencies installed successfully.")
except Exception as e:
    print(f"   ❌ FAILED to install system dependencies: {e}")
    raise

# 2. Clone or Update the TrinityUI control repository
print(f"\n[Step 2/3] Setting up TrinityUI control repository from '{TRINITY_BRANCH}' branch...")
clone_start = time.time()
try:
    if (PROJECT_ROOT / '.git').is_dir():
        print("   - Existing repository found. Updating...")
        # Using run_setup_command to see output if there are issues
        run_setup_command(f"git -c http.postBuffer=1048576000 --git-dir={PROJECT_ROOT / '.git'} --work-tree={PROJECT_ROOT} fetch")
        run_setup_command(f"git --git-dir={PROJECT_ROOT / '.git'} --work-tree={PROJECT_ROOT} reset --hard origin/{TRINITY_BRANCH}")
    else:
        print("   - Cloning repository with depth=1 for speed...")
        run_setup_command(f"git clone --depth 1 --branch {TRINITY_BRANCH} --single-branch {TRINITY_REPO_URL} {PROJECT_ROOT}")
    clone_time = time.time() - clone_start
    print(f"   ✅ TrinityUI repository ready in {clone_time:.2f}s.")
except Exception as e:
    print(f"   ❌ FAILED to setup repository: {e}")
    raise

# Modify manage_venvs.py to use more aggressive aria2c settings
print("\n[Step 2.5/3] Optimizing venv manager for maximum download speed...")
try:
    venv_manager_path = PROJECT_ROOT / 'scripts' / 'manage_venvs.py'
    if venv_manager_path.exists():
        # Read the file
        content = venv_manager_path.read_text()

        # Replace aria2c command with more aggressive parameters
        if "aria2c -c -x 16" in content:
            content = content.replace(
                "aria2c -c -x 16 -s 16 -k 1M -j $(nproc)",
                "aria2c -c -x 32 -s 32 -k 1M -j $(nproc) --min-split-size=1M --file-allocation=none --optimize-concurrent-downloads=true"
            )

            # Write the modified file
            venv_manager_path.write_text(content)
            print("   ✅ Venv manager optimized with aggressive download settings.")
        else:
            print("   ⚠️ Could not optimize aria2c settings in venv manager.")

        # Add a fix for the pip report parsing issue
        if "pip install --dry-run --report" in content:
            print("   🔄 Adding more robust URL extraction for aria2c acceleration...")
            # Replace the problematic function with a more reliable grep-based approach
            fixed_code = """
def fast_pip_install(venv_pip_path, requirements_file_path, cwd):
    log_message("Phase 1: Generating download URLs...")
    # Create a simpler method that doesn't rely on JSON parsing
    url_file = PIP_CACHE_DIR / "urls.txt"

    # Use pip download with dry run to see what would be downloaded
    cmd = f"{venv_pip_path} install --no-cache-dir --dry-run -r {requirements_file_path}"
    result = subprocess.run(cmd, shell=True, cwd=cwd, capture_output=True, text=True)

    # Extract URLs from the output
    urls = []
    for line in result.stdout.split('\\n'):
        if "Downloading" in line and "http" in line:
            url_part = line.split("Downloading ")[1].split()[0]
            if url_part.startswith("http"):
                urls.append(url_part)

    if urls:
        with open(url_file, 'w') as f:
            for url in urls:
                f.write(f"{url}\\n")

        log_message(f"Phase 2: Downloading {len(urls)} packages with aria2c...")
        aria_cmd = f"aria2c -c -x 32 -s 32 -k 1M -j $(nproc) --file-allocation=none --dir={PIP_CACHE_DIR} -i {url_file}"
        if not run_command_with_live_output(aria_cmd, cwd=PROJECT_ROOT):
            log_message("❌ aria2c download failed. Falling back to standard pip install.")
            return run_command_with_live_output(f"{venv_pip_path} install -r {requirements_file_path}", cwd=cwd)

        log_message("Phase 3: Installing packages from local cache...")
        pip_offline_cmd = f"{venv_pip_path} install --no-index --find-links={PIP_CACHE_DIR} -r {requirements_file_path}"
        return run_command_with_live_output(pip_offline_cmd, cwd=cwd)
    else:
        log_message("No download URLs found. Using standard pip install.")
        return run_command_with_live_output(f"{venv_pip_path} install -r {requirements_file_path}", cwd=cwd)
"""
            # Replace the existing function with our fixed version - using a simpler approach
            content = content.replace(
                "def fast_pip_install(venv_pip_path, requirements_file_path, cwd):",
                fixed_code
            )

            # Write the modified file
            venv_manager_path.write_text(content)
            print("   ✅ Added robust URL extraction for reliable downloads.")
    else:
        print("   ⚠️ Could not find venv manager script at expected location.")
except Exception as e:
    print(f"   ⚠️ Venv manager optimization failed: {e} (continuing anyway)")

# 3. Execute the Venv Manager
print("\n[Step 3/3] Executing the Venv Manager...")
print("="*60)
venv_start = time.time()
try:
    venv_manager_script = PROJECT_ROOT / 'scripts' / 'manage_venvs.py'

    # Set environment variables to maximize performance
    os.environ["PIP_DISABLE_PIP_VERSION_CHECK"] = "1"
    os.environ["PYTHONUNBUFFERED"] = "1"

    # Run with maximum thread count
    run_setup_command(f"python3 {venv_manager_script}")

    venv_time = time.time() - venv_start
    print("="*60)
    print(f"✅ All isolated WebUI environments have been set up in {venv_time:.2f}s!")

except Exception as e:
    print("="*60)
    print(f"❌ An error occurred during the Venv setup process: {e}")
    raise

# Set global variables for subsequent cells
globals()['PROJECT_ROOT'] = PROJECT_ROOT
globals()['WEBUI_ROOT'] = WEBUI_ROOT
globals()['trinity_setup_success'] = True

total_time = time.time() - clone_start
print(f"\n🎉 Infrastructure setup completed in {total_time:.2f}s. Proceed to Cell 2.")

🚀 Starting Trinity Multi-Venv Setup (Error-Resilient)...

[Step 1/3] Setting up system dependencies...
🚀 Executing: apt-get update -qq && apt-get install -y -qq aria2
  > W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
✅ Command successful. (5.60s)

   ✅ Aria2 download accelerator installed.
🚀 Executing: apt-get install -y -qq software-properties-common git curl wget
✅ Command successful. (2.36s)

🚀 Executing: add-apt-repository -y ppa:deadsnakes/ppa
  > Repository: 'deb https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu/ jammy main'
  > Description:
  > This PPA contains more recent Python versions packaged for Ubuntu.
  > 
  > Disclaimer: there's no guarantee of timely updates in case of security problems or other issues. If you want to use them in a security-or-otherwise-critical environment (say, on a production server), you do so at your 

In [ ]:
#@title 🎛️ **Cell 2: Project Trinity v1.1.0 - Configuration Hub** { display-mode: "form" }

# ============================================================================
# CELL 2: PROJECT TRINITY - CONFIGURATION HUB (FINAL LAUNCHER)
# This cell correctly launches the configuration hub as a background process
# and prints the public Gradio URL for access.
# ============================================================================

import os
import sys
import importlib.util
from pathlib import Path
from datetime import datetime
import socket
import time

# --- Helper Functions ---
def find_free_port(start_port=7860, max_attempts=20):
    """Find a free port starting from start_port"""
    for i in range(max_attempts):
        port = start_port + i
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            try:
                s.bind(('127.0.0.1', port))
                return port
            except OSError:
                continue
    return None

def log_to_unified(message, level="INFO", component="TRINITY-CELL-2"):
    """Log messages using the unified logging system from Cell 1"""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_file = globals().get('LOG_FILE', Path.cwd() / 'trinity_unified.log')
    version = globals().get('VERSION', '1.1.0')
    log_entry = f"[{timestamp}] [v{version}] [{level}] [{component}] {message}\n"
    try:
        with open(log_file, 'a', encoding='utf-8') as f:
            f.write(log_entry)
    except Exception as e:
        print(f"Warning: Could not write to log file: {e}")
    if level in ["ERROR", "SUCCESS"] or "Trinity" in message:
        print(f"[{level}] {message}")

# --- Infrastructure Check ---
def check_trinity_infrastructure():
    if 'PROJECT_ROOT' not in globals() or not globals()['PROJECT_ROOT'].exists():
        return False, "PROJECT_ROOT not found. Please run Cell 1 first."
    return True, "Infrastructure OK"

infrastructure_ok, infrastructure_msg = check_trinity_infrastructure()
if not infrastructure_ok:
    print(f"❌ CRITICAL: {infrastructure_msg}")
    raise RuntimeError(infrastructure_msg)

# --- Main Execution ---
PROJECT_ROOT = globals()['PROJECT_ROOT']
sys.path.insert(0, str(PROJECT_ROOT))

log_to_unified("Preparing Trinity Configuration Hub...")

# Find a free port
config_port = find_free_port(7860, 20)
if config_port is None:
    log_to_unified("Could not find a free port, using fallback 7861", "WARNING")
    config_port = 7861
else:
    log_to_unified(f"Found free port for Config Hub: {config_port}", "SUCCESS")

os.environ['TRINITY_CONFIG_PORT'] = str(config_port)

# Load and execute the configuration hub script
try:
    # Ensure we are using the corrected script
    config_script_path = PROJECT_ROOT / 'scripts' / 'configuration_hub_new.py'

    if config_script_path.exists():
        log_to_unified(f"Executing configuration hub from: {config_script_path}")

        # Use importlib to load the script as a module
        spec = importlib.util.spec_from_file_location("config_hub_module", config_script_path)
        config_hub_module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(config_hub_module)

        # Explicitly call the function that launches the Gradio app.
        # This will now print the URL and keep the cell running.
        if hasattr(config_hub_module, 'launch_trinity_configuration_hub'):
            log_to_unified("Launching Gradio interface...")
            # This function now contains the launch() call with share=True
            config_hub_module.launch_trinity_configuration_hub()
        else:
            raise RuntimeError("Could not find 'launch_trinity_configuration_hub' function in the script.")

    else:
        log_to_unified("Configuration Hub script (configuration_hub_new.py) not found!", "ERROR")
        print("❌ ERROR: scripts/configuration_hub_new.py not found.")

except Exception as e:
    log_to_unified(f"Configuration hub execution failed: {e}", "ERROR")
    print(f"❌ ERROR: Failed to launch the Configuration Hub. Details: {e}")
    raise


In [6]:
#@title 🚀 **Cell 3: Project Trinity v1.1.0 - Asset Download & Launch (With Link Detection)** { display-mode: "form" }

# ============================================================================
# CELL 3: PROJECT TRINITY - UNIFIED LAUNCH ENGINE
# This cell includes enhanced Gradio link detection
# ============================================================================

import os
import sys
import json
import subprocess
import re
import threading
import time
from pathlib import Path
from datetime import datetime
from IPython.display import HTML, display

# --- Helper Functions ---
def log_message(message, level="INFO"):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{level}] {message}")

def highlight_gradio_link(line):
    """Detects and highlights Gradio links in the output"""
    # Patterns for different types of Gradio URLs
    patterns = [
        r'(https?://[a-z0-9]+\.gradio\.live)',  # Standard Gradio share link
        r'(https?://[a-z0-9\-]+\.gradio\.app)',  # Alternative Gradio domain
        r'(Running on public URL: https?://[\w\.-]+)',  # A1111's Gradio message
        r'(Public WebUI URL: https?://[\w\.-]+)'        # Some variants of the message
    ]

    for pattern in patterns:
        match = re.search(pattern, line)
        if match:
            url = match.group(1)
            if 'http' not in url:
                url = re.search(r'(https?://[\w\.-]+)', url).group(1)

            # Display the URL prominently
            print("\n" + "="*80)
            print(f"🌐 GRADIO LINK DETECTED: {url}")
            print("="*80 + "\n")

            # Use IPython display for clickable link
            display(HTML(f'<div style="padding: 10px; background-color: #d1e7dd; border-radius: 5px; margin: 10px 0;">'
                         f'<h3>WebUI is ready! Click the link below to open:</h3>'
                         f'<a href="{url}" target="_blank" style="font-size: 16px; font-weight: bold;">{url}</a>'
                         f'</div>'))
            return True
    return False

# --- Infrastructure Check ---
PROJECT_ROOT = Path('/content/TrinityUI')
CONFIG_PATH = PROJECT_ROOT / "trinity_config.json"

if not PROJECT_ROOT.exists():
    log_message("PROJECT_ROOT not found. Please run Cell 1 first.", "ERROR")
    raise RuntimeError("PROJECT_ROOT not found. Please run Cell 1 first.")

# Check if configuration exists
if not CONFIG_PATH.exists():
    log_message(f"Configuration file not found at {CONFIG_PATH}", "ERROR")
    log_message("Please run Cell 2 and complete the configuration process in the Gradio interface.", "ERROR")
    raise FileNotFoundError(f"Configuration file not found at {CONFIG_PATH}")

# Load configuration to verify
try:
    with open(CONFIG_PATH, 'r') as f:
        config = json.load(f)
    webui_choice = config.get('webui_choice', 'Unknown')
    log_message(f"Configuration loaded successfully for {webui_choice}", "SUCCESS")
except Exception as e:
    log_message(f"Failed to load config file {CONFIG_PATH}: {e}", "ERROR")
    raise

# Add paths to Python path
sys.path.insert(0, str(PROJECT_ROOT))

print(f"✅ Infrastructure validated. Project root: {PROJECT_ROOT}")
print("   Preparing to execute the main launch engine...")

# Function to monitor for Gradio link in a separate thread
def watch_gradio_link(process, stop_event, webui_choice):
    """Constantly checks for a Gradio link in the process output"""
    try:
        log_message(f"Starting Gradio link monitoring for {webui_choice}...", "INFO")
        found_link = False

        while not stop_event.is_set() and process.poll() is None:
            try:
                # Read output line by line
                line = process.stdout.readline()
                if not line:
                    time.sleep(0.1)
                    continue

                # Check for Gradio links
                if 'gradio' in line.lower() or 'running on' in line.lower() or 'public url' in line.lower():
                    if highlight_gradio_link(line):
                        found_link = True
                        log_message(f"Found Gradio link for {webui_choice}!", "SUCCESS")

                # Don't consume CPU
                time.sleep(0.1)

            except Exception as e:
                log_message(f"Error in link monitoring: {e}", "ERROR")
                break

        if not found_link and not stop_event.is_set():
            log_message(f"Monitoring ended without finding a Gradio link for {webui_choice}", "WARNING")

    except Exception as e:
        log_message(f"Link monitoring thread crashed: {e}", "ERROR")

try:
    # Point to the execute_launch.py script
    script_path = PROJECT_ROOT / 'scripts' / 'execute_launch.py'

    if not script_path.exists():
        log_message(f"Launch script not found: {script_path}", "ERROR")
        raise FileNotFoundError(f"Could not find the main launch script: {script_path}")

    # Set environment variables for better performance
    env = os.environ.copy()
    env["PYTHONUNBUFFERED"] = "1"
    env["TRINITY_PROJECT_ROOT"] = str(PROJECT_ROOT)

    # Run the launch script
    process = subprocess.Popen(
        [sys.executable, str(script_path)],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
        env=env
    )

    # Start a thread to specifically look for Gradio links
    stop_monitoring = threading.Event()
    monitor_thread = threading.Thread(
        target=watch_gradio_link,
        args=(process, stop_monitoring, webui_choice)
    )
    monitor_thread.daemon = True
    monitor_thread.start()

    # Print a note about the link
    print("\n📝 NOTE: The Gradio link will be prominently displayed when it appears.")
    print("         It may take 2-5 minutes for the link to appear.")
    print("         Do not restart the cell during this time.\n")

    # Stream output to the notebook
    found_link_in_main = False
    for line in iter(process.stdout.readline, ''):
        sys.stdout.write(line)
        sys.stdout.flush()

        # Also check for Gradio links in the main thread
        if not found_link_in_main and ('gradio' in line.lower() or 'running on' in line.lower() or 'public url' in line.lower()):
            found_link_in_main = highlight_gradio_link(line)

    # Stop monitoring thread
    stop_monitoring.set()
    if monitor_thread.is_alive():
        monitor_thread.join(timeout=1)

    # Check final result
    return_code = process.poll()
    if return_code is not None and return_code != 0:
        log_message(f"Launch script exited with error code {return_code}.", "ERROR")
        print("\nYou can try a direct launch with this command in a new cell:")
        print(f"!cd /content/{webui_choice} && /content/{webui_choice}/venv/bin/python launch.py --share --xformers")

except KeyboardInterrupt:
    log_message("\n\n✅ Execution stopped by user.", "INFO")
    if 'process' in locals():
        process.terminate()
    if 'stop_monitoring' in locals():
        stop_monitoring.set()
except Exception as e:
    log_message(f"\n\n❌ An error occurred during execution: {e}", "ERROR")
    if 'process' in locals():
        process.terminate()
    if 'stop_monitoring' in locals():
        stop_monitoring.set()
    raise

[SUCCESS] Configuration loaded successfully for A1111
✅ Infrastructure validated. Project root: /content/TrinityUI
   Preparing to execute the main launch engine...
[INFO] Starting Gradio link monitoring for A1111...

📝 NOTE: The Gradio link will be prominently displayed when it appears.
         It may take 2-5 minutes for the link to appear.
         Do not restart the cell during this time.

[SUCCESS] Executing command: /content/A1111/venv/bin/python launch.py --api --xformers --no-half-vae

🚀 LAUNCHING A1111 FROM ITS VENV...
Python 3.10.12 (main, May 27 2025, 17:12:29) [GCC 11.4.0]
Version: v1.10.1
Commit hash: 82a973c04367123ae98bd9abdf80d9eda9b910e2
/content/A1111/venv/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Running on local URL:  http://127.0.0.1:7861

To cre

In [7]:
!cd /content/A1111 && /content/A1111/venv/bin/python launch.py --share --enable-insecure-extension-access --xformers --api --theme Default

Traceback (most recent call last):
  File "/content/A1111/launch.py", line 4, in <module>
    import matplotlib
  File "/content/A1111/venv/lib/python3.10/site-packages/matplotlib/__init__.py", line 1296, in <module>
    rcParams['backend'] = os.environ.get('MPLBACKEND')
  File "/content/A1111/venv/lib/python3.10/site-packages/matplotlib/__init__.py", line 771, in __setitem__
    raise ValueError(f"Key {key}: {ve}") from None
ValueError: Key backend: 'module://matplotlib_inline.backend_inline' is not a valid value for backend; supported values are ['gtk3agg', 'gtk3cairo', 'gtk4agg', 'gtk4cairo', 'macosx', 'nbagg', 'notebook', 'qtagg', 'qtcairo', 'qt5agg', 'qt5cairo', 'tkagg', 'tkcairo', 'webagg', 'wx', 'wxagg', 'wxcairo', 'agg', 'cairo', 'pdf', 'pgf', 'ps', 'svg', 'template']
